In [ ]:
import json
import uuid
from AI_Functions import call_llm
from RAG_Functions import create_collection

llm_model="gpt-3.5-turbo"
Real_Estate_Listings = create_collection("Real_Estate")




: 

In [ ]:
Real_Estate_Listings.count()

In [ ]:
# Define the function that will injects the list into the chroma DB
def generate_listing(Neighborhood, Price, Bedrooms, Bathrooms, House_Size, Description, Neighborhood_Description):
    print(Neighborhood, Price, Bedrooms, Bathrooms, House_Size, Description, Neighborhood_Description)
    unique_id = str(uuid.uuid4())
    collection.upsert(
    documents=[
        Description,Neighborhood_Description
    ],
    metadatas=[{"Neighborhood": Neighborhood, "Price": Price , "Bedrooms": Bedrooms, "Bathrooms": Bathrooms, "House_Size": House_Size},{"Neighborhood": Neighborhood, "Price": Price , "Bedrooms": Bedrooms, "Bathrooms": Bathrooms, "House_Size": House_Size}],
    ids=[f"{unique_id}_Description", f"{unique_id}_Neighborhood_Description"]
)


available_functions = {
        "generate_listing": generate_listing,
    }

In [ ]:
# Define the tools that will be available to the user
tools=[
    {
        "type": "function",
        "function": {
            "name": "generate_listing",
            "description": "Create a real estate Listing",
            "parameters": {
                "type": "object",
                "properties": {
                    "Neighborhood": {
                        "type": "string",
                        "description": "e.g. Green Oaks"
                    },
                    "Price": {
                        "type": "string",
                        "description": "e.g. $800,000"
                    },
                    "Bedrooms": {
                        "type": "number",
                        "description": "e.g. 2"
                    },
                    "Bathrooms": {
                        "type": "number"
                    },
                    "House Size": {
                        "type": "string",
                        "description": "e.g. 2,000 sqft"
                    },
                    "Description": {
                        "type": "string",
                        "description": "e.g. Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem."
                    },
                    "Neighborhood Description": {
                        "type": "string",
                        "description": "e.g. Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze"
                    }
                },
                "required": ["Neighborhood","Price","Bedrooms","Bathrooms","House Size","Description","Neighborhood Description"]
            }
        }
    }
    ]
messages = [{"role": "user", "content": "create 15 sample real estate listings"}]
response_message = call_llm(llm_model, messages, tools)
tool_calls = response_message.tool_calls



In [ ]:
tool_calls

In [ ]:
if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                Neighborhood=function_args.get("Neighborhood"),
                Price=function_args.get("Price"),
                Bedrooms=function_args.get("Bedrooms"),
                Bathrooms=function_args.get("Bathrooms"),
                House_Size=function_args.get("House Size"),
                Description=function_args.get("Description"),
                Neighborhood_Description=function_args.get("Neighborhood Description"),
            )
            print(function_response)

In [ ]:
collection.count()